<a href="https://colab.research.google.com/github/JanKrauland/Masterthesis_Jan-Krauland/blob/main/YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installieren und Laden benötigter Bibliotheken sowie des Git-Repositories



In [1]:
!git clone https://github.com/JanKrauland/Masterthesis_Jan-Krauland.git

Cloning into 'Masterthesis_Jan-Krauland'...
remote: Enumerating objects: 47022, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 47022 (delta 5), reused 11 (delta 4), pack-reused 47009
Receiving objects: 100% (47022/47022), 2.08 GiB | 17.68 MiB/s, done.
Resolving deltas: 100% (227/227), done.
Updating files: 100% (251210/251210), done.


In [2]:
!pip install ultralytics
#!pip uninstall albumentations -y #Diese Zeile wird nur beim initialen Training mit Standardparametern ausgeführt, um sicherzustellen, dass keine Data Augmentation erfolgt!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 8.1 MB/s eta 0:00:00


In [3]:
import datetime
import shutil
from pathlib import Path
from collections import Counter
import os
import yaml
import numpy as np
import pandas as pd
from ultralytics import YOLO
from sklearn.model_selection import KFold
import torch

In [4]:
# Prüfen, ob die GPU-Unterstützung aktiviert ist
if torch.cuda.is_available():
    print("CUDA verfügbar, Anzahl der GPUs:", torch.cuda.device_count())
else:
    print("CUDA nicht verfügbar")

CUDA verfügbar, Anzahl der GPUs: 1


In [ ]:
# Funktion zum Einfrieren von (10) Backbone-Schichten. Dient zum weiteren Testen von Transfer Learning
def freeze_layer(trainer):
    model = trainer.model
    num_freeze = 10
    print(f"Freezing {num_freeze} layers")
    freeze = [f'model.{x}.' for x in range(num_freeze)]  # layers to freeze
    for k, v in model.named_parameters():
        v.requires_grad = True  # train all layers
        if any(x in k for x in freeze):
            print(f'freezing {k}')
            v.requires_grad = False
    print(f"{num_freeze} layers are freezed.")

Training der Modelle mit K-Fold Cross-Validation

In [ ]:
results = {}

# Parameter des Trainings definieren. In diesem Fall wurde ein Versuch des Transfer Learnings unternommen.
batch = 32
project = "/content/drive/MyDrive/YOLO8m_K-Fold_100Epochs_Finetuning_Pretrained_10Layers_Freezed"
epochs = 100
ksplit = 5

for k in range(ksplit):
    model = YOLO("yolov8m.pt", task='detect')
    #model.add_callback("on_train_start", freeze_layer) #Aktivieren um Schichten einzufrieren

    path_to_yaml = "/content/Masterthesis_Jan-Krauland/Yolo_train/data_yamls/data_" + str(k+1) + ".yaml"
    dataset_yaml = path_to_yaml
    model.train(data=dataset_yaml, resume=False, augment=True, epochs=epochs, batch=batch, project=project)
    results[k] = model.metrics  # save output metrics for further analysis


Ultralytics YOLOv8.0.222 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/PlantVillage_on_YOLOv8m_50Epochs_Pretrain/train3/weights/best.pt, data=/content/drive/MyDrive/data_1.yaml, epochs=100, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/YOLO8m_K-Fold_100Epochs_Finetuning_Pretrained_10Layers_Freezed, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, reti

train: Scanning /content/PlantVillage_Tomato/K-Fold_PlantDoc/Split-1/train/labels.cache... 536 images, 0 backgrounds, 0 corrupt: 100%|██████████| 536/536 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/PlantVillage_Tomato/K-Fold_PlantDoc/Split-1/test/labels.cache... 135 images, 0 backgrounds, 0 corrupt: 100%|██████████| 135/135 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/YOLO8m_K-Fold_100Epochs_Finetuning_Pretrained_10Layers_Freezed/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Freezing 10 layers
freezing model.0.conv.weight
freezing model.0.bn.weight
freezing model.0.bn.bias
freezing model.1.conv.weight
freezing model.1.bn.weight
freezing model.1.bn.bias
freezing model.2.cv1.conv.weight
freezing model.2.cv1.bn.weight
freezing model.2.cv1.bn.bias
freezing model.2.cv2.conv.weight
freezing model.2.cv2.bn.weight
freezing model.2.cv2.bn.bias
freezing model.2.m.0.cv1.conv.weight
freezing model.2.m.0.cv1.bn.weight
freezing model.2.m.0.cv1.bn.bias
freezing model.2.m.0.cv2.conv.weight
freezing model.2.m.0.cv2.bn.weight
freezing model.2.m.0.cv2.bn.bias
freezing model.2

...

    100/100      5.14G      1.022      0.958      1.272        103        640: 100%|██████████| 17/17 [00:03<00:00,  4.28it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.13it/s]

                   all        135        545      0.294      0.372      0.313      0.172



100 epochs completed in 0.201 hours.
Optimizer stripped from /content/drive/MyDrive/YOLO8m_K-Fold_100Epochs_Finetuning_Pretrained_10Layers_Freezed/train5/weights/last.pt, 52.0MB
Optimizer stripped from /content/drive/MyDrive/YOLO8m_K-Fold_100Epochs_Finetuning_Pretrained_10Layers_Freezed/train5/weights/best.pt, 52.0MB

Validating /content/drive/MyDrive/YOLO8m_K-Fold_100Epochs_Finetuning_Pretrained_10Layers_Freezed/train5/weights/best.pt...
Ultralytics YOLOv8.0.222 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Model summary (fused): 218 layers, 25844392 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:05<00:00,  1.71s/it]


                   all        135        545      0.341      0.369       0.32      0.176
Tomato Early blight leaf        135         39      0.364       0.41      0.364      0.208
           Tomato leaf        135         74      0.407      0.432      0.403      0.204
Tomato leaf late blight        135         41      0.599      0.694      0.682      0.421
Tomato leaf bacterial spot        135         52      0.207      0.269      0.138     0.0832
Tomato leaf mosaic virus        135         45     0.0759     0.0667     0.0461     0.0263
Tomato leaf yellow virus        135        158      0.386      0.272      0.294       0.12
      Tomato mold leaf        135         55      0.269      0.291      0.213      0.115
Tomato Septoria leaf spot        135         81      0.422      0.519      0.422      0.233
Speed: 0.1ms preprocess, 23.3ms inference, 0.0ms loss, 9.8ms postprocess per image
Results saved to /content/drive/MyDrive/YOLO8m_K-Fold_100Epochs_Finetuning_Pretrained_10Layers_Freezed

Vortrainieren für Transfer Learning (Ohne K-Fold Cross-Validation)

In [ ]:
results = {}

batch = 32
project = "PlantVillage_on_YOLOv8m_50Epochs_Pretrain"
epochs = 10


model = YOLO("yolov8m.pt", task='detect')

path_to_yaml = "/content/data.yaml"

dataset_yaml = path_to_yaml
model.train(data=dataset_yaml, resume=False, augment=True, epochs=epochs, batch=batch, project=project)
results = model.metrics

Ultralytics YOLOv8.0.222 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/data.yaml, epochs=10, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=PlantVillage_on_YOLOv8m_50Epochs_Pretrain, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_b

train: Scanning /content/PlantVillage_Tomato/PlantVillage_Tomato/train_test_split/train/labels.cache... 12064 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12064/12064 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/PlantVillage_Tomato/PlantVillage_Tomato/train_test_split/test/labels.cache... 3016 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3016/3016 [00:00<?, ?it/s]


Plotting labels to PlantVillage_on_YOLOv8m_50Epochs_Pretrain/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to PlantVillage_on_YOLOv8m_50Epochs_Pretrain/train3
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      10/10      14.6G     0.0247     0.1091     0.8985         32        640: 100%|██████████| 377/377 [02:09<00:00,  2.92it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 48/48 [00:20<00:00,  2.33it/s]

                   all       3016       3016      0.983      0.982      0.994      0.994



10 epochs completed in 0.424 hours.
Optimizer stripped from PlantVillage_on_YOLOv8m_50Epochs_Pretrain/train3/weights/last.pt, 52.0MB
Optimizer stripped from PlantVillage_on_YOLOv8m_50Epochs_Pretrain/train3/weights/best.pt, 52.0MB

Validating PlantVillage_on_YOLOv8m_50Epochs_Pretrain/train3/weights/best.pt...
Ultralytics YOLOv8.0.222 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Model summary (fused): 218 layers, 25844392 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 48/48 [00:35<00:00,  1.35it/s]


                   all       3016       3016      0.986      0.988      0.994      0.994
Tomato Early blight leaf       3016        200      0.932      0.965      0.988      0.988
           Tomato leaf       3016        318      0.997      0.996      0.993      0.993
Tomato leaf late blight       3016        382      0.988       0.99      0.995      0.995
Tomato leaf bacterial spot       3016        425      0.979      0.995      0.995      0.995
Tomato leaf mosaic virus       3016         75          1      0.998      0.995      0.995
Tomato leaf yellow virus       3016       1071          1      0.997      0.995      0.995
      Tomato mold leaf       3016        191       0.99      0.995      0.995      0.995
Tomato Septoria leaf spot       3016        354      0.999      0.972      0.995      0.995
Speed: 0.1ms preprocess, 6.9ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to PlantVillage_on_YOLOv8m_50Epochs_Pretrain/train3


Evaluation

In [6]:
for i in range(5):
    if i == 0:
        model = YOLO('/content/drive/MyDrive/YOLO8m_K-Fold_50Epochs_Augmentation/train/weights/best.pt')  # Laden des besten Modells für den ersten Split


        # Validieren des Modells
        metrics = model.val(data='/content/Masterthesis_Jan-Krauland/Yolo_train/data_yamls/data_1.yaml', split='test')
        metrics.box.map    # map50-95
        metrics.box.map50  # map50
        metrics.box.map75  # map75
        metrics.box.maps
        continue

    model = YOLO('/content/drive/MyDrive/YOLO8m_K-Fold_50Epochs_Augmentation/train' + str(i+1) + '/weights/best.pt')  # Laden des besten Modells für jeden weiteren Split


    # Validieren des Modells
    metrics = model.val(data='/content/Masterthesis_Jan-Krauland/Yolo_train/data_yamls/data_' + str(i+1) + '.yaml', split='test')
    metrics.box.map    # map50-95
    metrics.box.map50  # map50
    metrics.box.map75  # map75
    metrics.box.maps

Ultralytics YOLOv8.0.228 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Model summary (fused): 218 layers, 25844392 parameters, 0 gradients, 78.7 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 39.5MB/s]
val: Scanning /content/Masterthesis_Jan-Krauland/PlantDoc_Test/test/labels... 69 images, 0 backgrounds, 0 corrupt: 100%|██████████| 69/69 [00:00<00:00, 961.00it/s]

val: New cache created: /content/Masterthesis_Jan-Krauland/PlantDoc_Test/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:06<00:00,  1.34s/it]


                   all         69        192      0.539      0.414      0.482      0.345
Tomato Early blight leaf         69         19       0.59      0.474      0.551       0.41
           Tomato leaf         69         27      0.467      0.333      0.437      0.326
Tomato leaf late blight         69         14      0.585      0.786      0.787      0.616
Tomato leaf bacterial spot         69         14      0.402      0.286      0.281      0.236
Tomato leaf mosaic virus         69         36      0.691      0.111      0.332      0.198
Tomato leaf yellow virus         69         42      0.788      0.548      0.645      0.365
      Tomato mold leaf         69         16      0.233       0.25      0.257      0.192
Tomato Septoria leaf spot         69         24      0.558      0.526      0.563      0.419
Speed: 2.1ms preprocess, 38.9ms inference, 0.0ms loss, 22.6ms postprocess per image
Results saved to runs/detect/val
Ultralytics YOLOv8.0.228 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (

...

                   all         69        192      0.488      0.537      0.527      0.356
Tomato Early blight leaf         69         19      0.587      0.524      0.588      0.416
           Tomato leaf         69         27      0.298       0.37      0.287      0.191
Tomato leaf late blight         69         14      0.517      0.786       0.69      0.531
Tomato leaf bacterial spot         69         14      0.228        0.5      0.385      0.295
Tomato leaf mosaic virus         69         36      0.742      0.306       0.54      0.345
Tomato leaf yellow virus         69         42      0.591      0.667      0.693      0.346
      Tomato mold leaf         69         16      0.316      0.438      0.457      0.287
Tomato Septoria leaf spot         69         24      0.627      0.708      0.578      0.434
Speed: 2.6ms preprocess, 14.8ms inference, 0.0ms loss, 19.4ms postprocess per image
Results saved to runs/detect/val3
Ultralytics YOLOv8.0.228 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 

val: Scanning /content/Masterthesis_Jan-Krauland/PlantDoc_Test/test/labels.cache... 69 images, 0 backgrounds, 0 corrupt: 100%|██████████| 69/69 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:04<00:00,  1.03it/s]


                   all         69        192      0.432      0.615      0.572       0.39
Tomato Early blight leaf         69         19      0.493      0.632      0.599      0.449
           Tomato leaf         69         27      0.227       0.37       0.38      0.256
Tomato leaf late blight         69         14      0.447      0.786      0.845      0.611
Tomato leaf bacterial spot         69         14      0.315      0.643      0.468      0.379
Tomato leaf mosaic virus         69         36      0.692      0.306      0.521      0.337
Tomato leaf yellow virus         69         42      0.531      0.667      0.633      0.315
      Tomato mold leaf         69         16      0.327      0.812      0.544      0.367
Tomato Septoria leaf spot         69         24      0.422      0.708      0.585      0.407
Speed: 2.4ms preprocess, 12.3ms inference, 0.0ms loss, 36.8ms postprocess per image
Results saved to runs/detect/val4
Ultralytics YOLOv8.0.228 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 

val: Scanning /content/Masterthesis_Jan-Krauland/PlantDoc_Test/test/labels.cache... 69 images, 0 backgrounds, 0 corrupt: 100%|██████████| 69/69 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:05<00:00,  1.01s/it]


                   all         69        192      0.465       0.52      0.502      0.344
Tomato Early blight leaf         69         19      0.417      0.526        0.4      0.285
           Tomato leaf         69         27      0.407      0.444      0.474      0.321
Tomato leaf late blight         69         14      0.531      0.857      0.875      0.674
Tomato leaf bacterial spot         69         14      0.422        0.5      0.339      0.246
Tomato leaf mosaic virus         69         36      0.625      0.417      0.497      0.306
Tomato leaf yellow virus         69         42      0.681      0.643      0.716       0.38
      Tomato mold leaf         69         16      0.239      0.312      0.269      0.206
Tomato Septoria leaf spot         69         24      0.394      0.458      0.442      0.332
Speed: 3.1ms preprocess, 6.6ms inference, 0.0ms loss, 12.8ms postprocess per image
Results saved to runs/detect/val5
